In [ ]:
from GBP.data import DataGenerator
from GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_COARSE
from GBP.utilities import HiddenPrints
from GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult

import warnings
import matplotlib
import numpy as np
import math
import random

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

In [ ]:
import random

def custom_sample(items, n):
    indices = random.sample(range(len(items)), n)
    return [items[i] for i in indices]

# Example usage
items = ["apple", "banana", "orange", "grape", "kiwi"]
n = 3
selected_items = custom_sample(items, n)
print(selected_items)

In [ ]:
num_nodes = 10

A, b = data_gen.get_1D_line_matrix(num_nodes, scaling=True, normalized=False)
# A, b = data_gen.get_2D_lattice_matrix(num_nodes, num_nodes)
graph = NetworkxGraph(A)
# graph.draw_graph()

sync_convergence_threshold = 1*10**-5

#### Sync

In [ ]:
sum_of_iterations = 0
num_iterations = 1

for it in range(0,num_iterations):
    print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=100000, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
    sum_of_iterations += iteration
    final_mean = list(mu_i)
    final_std = P_i

print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Async Random - No Capping

In [ ]:
# async_convergence_threshold = 1*10**-2

# sum_of_iterations = 0
# num_iterations = 100

# for it in range(0,num_iterations):
#     print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, node_updates_per_pe=1, number_pes=num_nodes//2, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations += iteration

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Async Random - Capping to 1 Factor Used

In [ ]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = 100

for it in range(0,num_iterations):
    print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_COARSE(A, b, node_updates_per_pe=1, number_pes=num_nodes//2, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Fixed - No Capping

In [ ]:
# async_convergence_threshold = 1*10**-2

# sum_of_iterations = 0
# num_iterations = 100

# for it in range(0,num_iterations):
#     print(f"-------------- ITERATION = {it+1} --------------")
#     node_update_schedule = np.arange(num_nodes, dtype=np.int64)
#     np.random.shuffle(node_update_schedule)
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=1, number_pes=num_nodes//2, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations += iteration

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Fixed - Capping to 1 Factor Used

In [ ]:
# async_convergence_threshold = 1*10**-2

# sum_of_iterations = 0
# num_iterations = 1000

# for it in range(0,num_iterations):
#     print(f"-------------- ITERATION = {it+1} --------------")
#     node_update_schedule = np.arange(num_nodes, dtype=np.int64)
#     np.random.shuffle(node_update_schedule)
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, capping=1, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=1, number_pes=num_nodes//2, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations += iteration

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [ ]:
# async_convergence_threshold = 1*10**-2

# sum_of_iterations_capping_no_shuffle = 0
# sum_of_iterations_capping_shuffle = 0

# sum_of_iterations_no_capping_no_shuffle = 0
# sum_of_iterations_no_capping_shuffle = 0

# num_iterations = 10000

# for it in range(0,num_iterations):
#     print(f"-------------- ITERATION = {it+1} --------------")
#     node_update_schedule = np.arange(num_nodes, dtype=np.int64)
#     np.random.shuffle(node_update_schedule)
    
#     # CAPPING (NO SHUFFLED)
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, capping=1, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=1, number_pes=num_nodes//2, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations_capping_shuffle += iteration
    
#     # CAPPING (SHUFFLED)
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, shuffled_fixed=False, capping=1, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=1, number_pes=num_nodes//2, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations_capping_no_shuffle += iteration

#     # NO CAPPING (NO SHUFFLED)
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, capping=2, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=1, number_pes=num_nodes//2, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations_no_capping_shuffle += iteration
    
#     # NO CAPPING (NO SHUFFLED)
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, shuffled_fixed=False, capping=2, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=1, number_pes=num_nodes//2, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations_no_capping_no_shuffle += iteration


# print(f"AVE. ASYNC ITERATIONS (Capping & No Shuffle) = {sum_of_iterations_capping_no_shuffle/num_iterations}")
# print("----------")
# print(f"AVE. ASYNC ITERATIONS (Capping & Shuffle) = {sum_of_iterations_capping_shuffle/num_iterations}")
# print("----------")
# print(f"AVE. ASYNC ITERATIONS (No Capping & No Shuffle) = {sum_of_iterations_no_capping_no_shuffle/num_iterations}")
# print("----------")
# print(f"AVE. ASYNC ITERATIONS (No Capping & Shuffle) = {sum_of_iterations_no_capping_shuffle/num_iterations}")

### Test

In [ ]:
# from numba import njit
# from numba.typed import List

# @njit(fastmath=True)
# def assign_values_to_pes_fixed(node_update_schedule, 
#                                 node_updates_per_pe, 
#                                 number_pes, 
#                                 current_index):
#     # select number of nodes
#     total_num_nodes = number_pes * node_updates_per_pe
    
#     # Get the current index to start from
#     start_index = current_index
    
#     # Calculate end index
#     end_index = (start_index + total_num_nodes) % len(node_update_schedule)

#     # Handle wrap-around case
#     if end_index > start_index:
#         values_pool = node_update_schedule[start_index:end_index]
#     else:
#         values_pool = node_update_schedule[start_index:] + node_update_schedule[:end_index]

#     # Distribute values among PEs
#     PEs = {}
#     for i in range(number_pes):
#         start_index = i * node_updates_per_pe
#         end_index = min((i + 1) * node_updates_per_pe, total_num_nodes)
#         pe_values = values_pool[start_index:end_index]
#         # Ensure all values are integers
#         pe_values = List(pe_values)
#         PEs[i] = pe_values

#     # Update current index for next call
#     current_index = (current_index + total_num_nodes) % len(node_update_schedule)
#     return PEs, current_index

# # Example usage:
# node_update_schedule = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# node_updates_per_pe = 1
# number_pes = 1
# current_index = 0

# # Execute the function several times to demonstrate behavior
# for _ in range(5):
#     pes, current_index = assign_values_to_pes_fixed(
#         node_update_schedule=node_update_schedule,
#         node_updates_per_pe=node_updates_per_pe,
#         number_pes=number_pes,
#         current_index=current_index
#     )
#     print(pes)

In [ ]:
# pes, current_index = assign_values_to_pes_fixed(node_update_schedule=node_update_schedule, node_updates_per_pe=node_updates_per_pe, number_pes=number_pes, current_index=current_index)
# print(pes)

# pes, current_index = assign_values_to_pes_fixed(node_update_schedule=node_update_schedule, node_updates_per_pe=node_updates_per_pe, number_pes=number_pes, current_index=current_index)
# print(pes)